In [87]:
import ee
import folium
import geemap as emap

In [2]:
# Initialize ee
ee.Initialize()
ee.Authenticate()

Enter verification code: 4/1AX4XfWiP3RW2tRpxyBf0dePjTXowIS9epgl10YiIV8bFzfxFNADz3LQYxCg

Successfully saved authorization token.


In [95]:
Map=emap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

# 1. Select Images

##### Filter or get `best` image from a ImageCollection

In [114]:
# Filter date, bounds and sort by cloud cover
## Define the region of interest
point=ee.Geometry.Point(105.46,21.41)
## Filter by collection
collection=ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate("2014-01-01","2015-01-01").\
filterBounds(point).sort("CLOUD_COVER")

In [115]:
# Print out the number of images 
collection.size().getInfo()

17

In [116]:
# Get the best image
image=collection.first()

In [119]:
# Visualizing the above image
vis={"bands":["B4","B3","B2"],"min":0.046, "max":0.389}
# Center the map
Map.centerObject(point,10)
# Plot the image
Map.addLayer(image, vis, "Image 2014")

##### Write a function to select the best image by each year from 2015 to 2020

In [76]:
def bestImage(year):
    start_year=ee.Date.fromYMD(year, 1,1)
    end_year=start_year.advance(1,"year")
    image=ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate(start_year,end_year).\
filterBounds(point).sort("CLOUD_COVER").first()
    return image
years=ee.List.sequence(2015,2020)

myImage=years.map(bestImage)

In [81]:
collection=ee.ImageCollection(myImage)
# See cloud cover each image
for i in collection.aggregate_array("CLOUD_COVER").getInfo():
    print("Cloud Cover: {:.3} %".format(i))

Cloud Cover: 19.1 %
Cloud Cover: 1.53 %
Cloud Cover: 5.54 %
Cloud Cover: 8.37 %
Cloud Cover: 0.12 %
Cloud Cover: 25.1 %


In [120]:
# Visualize each image
for index in range(0,6):
    image=ee.Image(myImage.get(index))
    layerName="Image "+ str(years.getInfo()[index])
    Map.addLayer(image, vis, layerName, False)